# XGBoost

# Imports

In [1]:
!pip install pandas==1.4.2
!pip install matplotlib==3.5.2
!pip install seaborn==0.11.2
!pip install nltk==3.7
!pip install scikit-learn==1.1.1
!pip install xgboost==1.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.4.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.12.2
    Uninstalling seaborn-0.12.2:
      Successfully uninstalled seaborn-0.12.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.7.6
    Uninstalling xgboost-1.7.6:
      Successfully uninstalled xgboost-1.7.6


In [2]:
import pandas as pd

In [3]:
# Change the file directories to yours, the files are in a public folder in google drive:
# https://drive.google.com/drive/folders/1CsvJs0xZ9SCLtG4ci2dNGCkhqnI-1DLS?usp=drive_link
df_train_valid = pd.read_parquet('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/train')
df_test = pd.read_parquet('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/test')

In [4]:
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

In [5]:
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from sklearn.impute import SimpleImputer

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [8]:
from sklearn.model_selection import RandomizedSearchCV

# Agrego valores a una columna vacía

In [9]:
df_train_valid['content'] = df_train_valid['content'].fillna(value='')
df_test['content'] = df_test['content'].fillna(value='')

In [10]:
count_vectorizer = CountVectorizer(lowercase=True, max_features=5000, stop_words='english')

array = count_vectorizer.fit_transform(df_train_valid['content'].tolist()).toarray()
df_train_valid['n_non_stop_words'] = [sum(arr) for arr in array]

array = count_vectorizer.transform(df_test['content'].tolist()).toarray()
df_test['n_non_stop_words'] = [sum(arr) for arr in array]

# Separo el dataframe de train en: train y valid

In [11]:
#Ordeno el df de train_valid
df_train_valid = df_train_valid.sort_values('timedelta', ascending=False)
#Separo el df de train en train y validation
df_train = df_train_valid.iloc[:89*(len(df_train_valid)//100)].copy()
df_valid = df_train_valid.iloc[89*(len(df_train_valid)//100):].copy()

# Encoding

Encodeo surprise2 con MeanEncoding

In [12]:
df_train['surprise2'] =  df_train['surprise2'].map(df_train.groupby(['surprise2'])['popular'].mean().to_dict())
df_valid['surprise2'] =  df_valid['surprise2'].map(df_valid.groupby(['surprise2'])['popular'].mean().to_dict())
df_test['surprise2'] =  df_test['surprise2'].map(df_test.groupby(['surprise2'])['popular'].mean().to_dict())

Encodeo surprise1 con OneHotEncoder

In [13]:
ohe_surp1 = OneHotEncoder()

surp1_train = ohe_surp1.fit_transform(df_train['surprise1'].to_numpy().reshape(-1,1)).todense().astype(int)
surp1_valid = ohe_surp1.transform(df_valid['surprise1'].to_numpy().reshape(-1,1)).todense().astype(int)
surp1_test = ohe_surp1.transform(df_test['surprise1'].to_numpy().reshape(-1,1)).todense().astype(int)

df_train[list(ohe_surp1.categories_[0])] = surp1_train
df_valid[list(ohe_surp1.categories_[0])] = surp1_valid
df_test[list(ohe_surp1.categories_[0])] = surp1_test

Encodeo content con CountVectorizer

In [14]:
count_vectorizer = CountVectorizer(lowercase=True, max_features=10, stop_words='english')

In [15]:
dataframes = [df_train, df_valid, df_test]

In [16]:
for index, df in enumerate(dataframes):

  if index == 0:
    array = count_vectorizer.fit_transform(df['content'].tolist()).toarray()
  else:
    array = count_vectorizer.transform(df['content'].tolist()).toarray()

  names = count_vectorizer.get_feature_names_out()

  for index, name in enumerate(names):
    df[name] = [arr[index] for arr in array]

# Separo dfs en x e y

In [17]:
#Dropeo los features que no me sirven
x_train_df = df_train.drop(['shares', 'popular', 'url', 'timedelta', 'content', 'surprise1'], axis=1, inplace=False)
y_train = df_train['popular']

x_valid_df = df_valid.drop(['shares', 'popular', 'url', 'timedelta', 'content', 'surprise1'], axis=1, inplace=False)
y_valid = df_valid['popular']

x_test_df = df_test.drop(['shares', 'popular', 'url', 'timedelta', 'content', 'surprise1'], axis=1, inplace=False)
y_test = df_test['popular']

# Reemplazo los NaN

In [18]:
imputer = SimpleImputer(strategy='mean', copy=False)

x_train = imputer.fit_transform(x_train_df)
x_valid = imputer.transform(x_valid_df)
x_test = imputer.transform(x_test_df)

# Modelo base

In [19]:
model = xgb.XGBClassifier()

In [20]:
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

# Modelo con mejores hiperparámetros

In [27]:
model = xgb.XGBClassifier(alpha=5, n_estimators =25)

In [28]:
model.fit(x_train, y_train)

XGBClassifier(alpha=5, base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=25, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=5, ...)

# Modelo con búsqueda de hiperparámetros

In [23]:
model = xgb.XGBClassifier()

In [24]:
params = {
    'n_estimators ': [10, 15, 20, 25],
    'max_depth': [2, 3, 4, 5, 6],
    'learning_rate': [0.02, 0.05, 0.1],
    'alpha': [5, 10, 15]
}

In [25]:
model = RandomizedSearchCV(model, params, n_iter=25, random_state=10)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
model = model.best_estimator_

# Resultados

In [29]:
binary_preds = model.predict(x_valid)
proba_preds = model.predict_proba(x_valid)[:,1]

In [30]:
print(classification_report(y_valid, binary_preds))

              precision    recall  f1-score   support

       False       0.83      0.99      0.90      3291
        True       0.53      0.06      0.10       688

    accuracy                           0.83      3979
   macro avg       0.68      0.52      0.50      3979
weighted avg       0.78      0.83      0.77      3979



In [31]:
roc_auc_score(y_valid, proba_preds)

0.7317715510235808

# Genero csv

In [ ]:
proba_preds_test = model.predict_proba(x_test)[:,1]

In [ ]:
df_preds = pd.DataFrame(proba_preds_test, columns=['popular prediction'])
df_preds['url'] = df_train['url']
df_preds = df_preds[['url', 'popular prediction']]

In [ ]:
df_preds.to_csv('preds_test_p3.csv')